# Pretraining a Model on Twitter Sentiment

In [2]:
import pandas as pd
import numpy as np

## Building a Words List and Word Vector matrix

We need to include Trump's vocab in both the wordlist and the Word2Vec matrix so that we can make predictions on it.

### Wordlist

In [3]:
df = pd.read_csv("./data/random_tweets_sentiment.csv", encoding="latin1", usecols=[1, 2])
df.rename(columns={"Sentiment":"sentiment", "SentimentText":"text"}, inplace=True)
df.head()

FileNotFoundError: File b'./data/random_tweets_sentiment.csv' does not exist

In [129]:
df2 = pd.read_csv("./data/alltweets.csv", encoding="latin1", usecols=[0])
print(df2.shape)
df2.head()

FileNotFoundError: File b'./data/alltweets.csv' does not exist

In [130]:
sentences = df.text.append(df2.text)
sentences.head()

NameError: name 'df' is not defined

In [131]:
sentences.shape

NameError: name 'sentences' is not defined

In [132]:
sentences.isna().sum()

NameError: name 'sentences' is not defined

In [133]:
sentences = sentences.values
len(sentences)

NameError: name 'sentences' is not defined

In [134]:
import re

wordlist = list(set(re.split("( |[^a-zA-Z\d])", "".join(sentences).lower())))
#wordlist = np.array(wordlist)

NameError: name 'sentences' is not defined

In [135]:
# import pickle 

# file = open("./data/tweets_wordlist.pkl", "wb")
# pickle.dump(wordlist, file)

In [136]:
# f = pickle.load(open("./data/tweets_wordlist.pkl", "rb"))

# for x in f:
#     print(x)

In [137]:
len(wordlist)
#wordlist.shape

NameError: name 'wordlist' is not defined

In [138]:
# import pickle 

# file = open("./data/tweets_edited.pkl", "wb")
# pickle.dump(sentences, file)

### WordVector

First we swap words in the words matrix for the indices in the wordlist. We can then use this integer matrix as input to the Word2Vec algorithm.

In [139]:
# Running this on a GPU

# tweets = []
# sizes_sum = 0

# for i in range(len(sentences)):
#     s_lower_split = re.split("( |[^a-zA-Z\d])", "".join(sentences[i]).lower())
#     s_lower_split = list(filter(lambda x : x not in [" ", ""], s_lower_split))
#     s_lower_split = list(map(lambda x : wordlist.index(x), s_lower_split))
#     sizes_sum += len(s_lower_split)
#     tweets.append(s_lower_split)
#     print(i)

In [ ]:
# print(model)

In [ ]:
model.build_vocab

## Building the Word2Vec model

In [44]:
import tensorflow as tf

In [45]:
batch_size = 50
tweets = np.load("data/tweets_integerized.npy")
tweets = [item for sublist in tweets for item in sublist]

couples, labels = tf.keras.preprocessing.sequence.skipgrams(tweets, len(tweets), window_size=1)
bigrams = [x for x, y in zip(couples, labels) if y]
print(bigrams)
batches = [bigrams[i:i + batch_size] for i in range(0, len(bigrams), batch_size)]
print("batches")
print(batches[0])
input_inputs = [list(zip(*x)) for x in batches]

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



batches
[[53416, 65546], [26671, 120680], [20350, 4491], [55460, 68109], [100843, 104062], [15792, 5223], [57511, 23092], [57633, 32620], [46947, 34007], [69342, 82278], [5856, 66215], [121806, 46947], [2200, 72480], [75192, 21165], [76280, 120680], [3204, 108065], [88683, 28110], [76280, 111853], [28666, 28666], [105781, 23342], [62652, 130940], [6184, 64931], [120680, 115779], [120680, 120680], [133963, 54040], [121701, 57633], [120680, 96378], [100843, 48175], [54965, 120680], [2200, 100843], [121806, 95263], [103027, 119585], [130259, 62652], [106080, 120680], [76280, 107225], [98233, 108839], [128265, 76001], [76001, 120680], [77979, 69388], [83922, 131679], [69453, 128368], [87897, 120680], [69996, 28110], [23342, 109281], [100757, 60173], [108839, 23092], [91826, 127952], [51951, 69388], [114299, 57511], [115210, 11303]]


In [46]:
import math
num_sampled = 64
graph = tf.Graph()
vocab_size = len(tweets)
embedding_size = 4
with graph.as_default():
    
    embeddings = tf.Variable(tf.random_uniform([vocab_size, embedding_size], -1.0, 1.0))
    
    nce_weights = tf.Variable(tf.truncated_normal(
        [vocab_size, embedding_size], stddev=1.0/math.sqrt(embedding_size)))
    
    nce_biases = tf.Variable(tf.zeros([vocab_size]))
    
    
    train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
    train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
    
    embed = tf.nn.embedding_lookup(embeddings, train_inputs)
    
    loss = tf.reduce_mean(
        tf.nn.nce_loss(weights=nce_weights,
                       biases=nce_biases,
                       labels=train_labels,
                       inputs=embed,
                       num_sampled=num_sampled,
                       num_classes=vocab_size))
    
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=1.0).minimize(loss)
    init = tf.global_variables_initializer()

In [ ]:

with tf.Session(graph=graph) as session:
    init.run()
    
    for inputs, labels in input_inputs:
        print("batch")
        inputs = np.array(inputs)
        labels_array = [[l] for l in labels]
        print(inputs)
        print(labels_array)
        feed_dict = {train_inputs: inputs, train_labels: labels_array}
        _, cur_loss = session.run([optimizer, loss], feed_dict=feed_dict)

print("yeet done")

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.python.tools import inspect_checkpoint as chkp
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

tf.reset_default_graph()

# Create some variables.
v1 = tf.get_variable("Variable", shape=[2868364,4])

# Add ops to save and restore all the variables.
saver = tf.train.Saver({"Variable": v1})

# Later, launch the model, use the saver to restore variables from disk, and
# do some work with the model.
with tf.Session() as sess:
  # Restore variables from disk.
  saver.restore(sess, "/tmp/model.ckpt")
  print("Model restored.")
  # Check the values of the variables
  print("v1 : %s" % v1.eval())

  wordVectors = v1.eval()

embedded = TSNE(n_components=2, perplexity=50).fit_transform(final_embeddings[:1000])

plt.scatter(embedded[:, 0], embedded[:, 1])
plt.show()

NameError: name 'wordList' is not defined